In [49]:
import numpy as np
from astropy import units as u, constants as const
import pandas as pd

In [50]:
def input_ready(r_star, t_star, a_au):
    #Converts inputs to relevant SI units & loading constansts
    R_Star = (r_star * const.R_sun).to(u.m) #!
    T_Star = t_star * u.K
    a = (a_au * u.au).to(u.m) #!
    return R_Star, T_Star, a
    

In [51]:
def compute_sl(r_star, t_star):
    #Computing Stellar luminosity 
    sigma = const.sigma_sb 
    L_star = 4 * np.pi * r_star**2 * sigma * t_star**4
    return L_star
    

In [52]:
def compute_teq(r_star, t_star, a_au, albedo = 0.3):
    #Computes Equilibrium Temp
    R_Star, T_Star, a = input_ready(r_star, t_star, a_au)
    L_Star = compute_sl(R_Star, T_Star)

    sigma = const.sigma_sb 
    numerator = (1 - albedo) * L_Star
    denominator = 16 * np.pi * sigma * a**2
    T_eq = ((numerator/denominator) ** 0.25 ).to(u.K)

    return T_eq

    

In [53]:
T_eq = compute_teq(1, 5778, 1, albedo = 0.3)
print(f"{T_eq:.2f}")

254.85 K


In [57]:
sample = pd.read_csv("raw_data/exoplanet_sample.csv")
teq_list = []

for idx, row in sample.iterrows():
    Teq = compute_teq(row["st_rad"], row["st_teff"], row["pl_orbsmax"], albedo = 0.3)
    teq_list.append(Teq.value)

sample["Teq"] = teq_list
sample.loc[sample["Teq"] > 800, 'type'] = 'hotter'
print((sample['type'] == 'hotter').sum())
sample.to_csv("raw_data/exo_teq_results.csv", index = False)
sample
    

493


,pl_name,pl_orbsmax,pl_orbsmaxerr1,pl_orbsmaxerr2,pl_orbsmaxlim,st_rad,st_raderr1,st_raderr2,st_radlim,st_teff,st_tefferr1,st_tefferr2,st_tefflim,type,Teq
0,Kepler-645 b,0.04420,NaN,NaN,0.0,1.27300,0.664000,-0.227000,0.0,6449.0,173.00,-227.00,0.0,hotter,1526.522648
1,TOI-3919 b,0.07950,0.00140,-0.00160,0.0,1.31900,0.052000,-0.048000,0.0,6100.0,110.00,-110.00,0.0,hotter,1095.914003
2,Kepler-380 c,0.07580,NaN,NaN,0.0,1.19100,0.186000,-0.186000,0.0,6038.0,121.00,-121.00,0.0,hotter,1055.655451
3,Kepler-410 A b,0.12260,0.00470,-0.00470,0.0,1.42749,0.056457,-0.082478,0.0,6304.0,156.96,-125.63,0.0,hotter,948.779548
4,HATS-26 b,0.04735,0.00133,-0.00068,0.0,2.04000,0.150000,-0.110000,0.0,6071.0,81.00,-81.00,0.0,hotter,1757.612501
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
765,HD 171238 b,2.57000,0.16000,-0.16000,0.0,1.01000,0.070000,-0.070000,0.0,5440.0,NaN,NaN,0.0,cold,150.418354
766,kap CrB b,2.70000,NaN,NaN,0.0,4.83752,NaN,NaN,0.0,4871.0,25.00,-25.00,0.0,cold,287.577856
767,TOI-2537 c,2.78000,0.22000,-0.15000,0.0,0.77100,0.039000,-0.040000,0.0,4870.0,150.00,-150.00,0.0,cold,113.120564
768,HD 106270 b,4.30000,0.40000,-0.40000,0.0,2.50000,0.100000,-0.100000,0.0,5638.0,44.00,-44.00,0.0,cold,189.613239
